In [8]:
import os
import re
import json
import uuid
from pathlib import Path

import pandas as pd
import numpy as np

In [3]:
fp = '/diskmnt/Projects/Users/estorrs/sandbox/specimens.txt'
df = pd.read_csv(fp, sep='\t')
df

,run_name,omero_run_name,date,case,specimen,bbox,filepath
0,20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1,2024-08-12_HT913Z1,20240812,HT913Z1,HT913Z1-S2H1FP1_U2,"562,14052,0,16950",/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...
1,20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1,2024-08-12_HT913Z1,20240812,HT913Z1,HT913Z1-S2H1FP1_U12,"18424,31915,774,17324",/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...
2,20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1,2024-08-12_HT913Z1,20240812,HT913Z1,HT913Z1-S2H1FP1_U22,"33663,47341,649,17012",/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...
3,20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1,2024-08-12_HT913Z1,20240812,HT913Z1,HT913Z1-S2H1FP1_U41,"49777,63330,3084,19261",/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...
4,20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_2,2024-08-12_HT913Z1_2,20240812,HT913Z1,HT913Z1-S2H1FP1_U51,"589,13911,404,16732",/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...
...,...,...,...,...,...,...,...
346,20211019-HT339B1-S1H3A1,2021-10-19_HT339B1,20211019,HT339B1,HT339B1-S1H3A1,NaN,/diskmnt/primary/CODEX/HTAN/20211019-HT339B1-S...
347,20211008_HT323B1-S1H3A4,2021-10-08_HT323B1,20211008,HT323B1,HT323B1-S1H3A4,NaN,/diskmnt/primary/CODEX/HTAN/20211008_HT323B1-S...
348,20210921_HT293N1-S1H3A1,2021-09-21_HT293N1,20210921,HT293N1,HT293N1-S1H3A1,NaN,/diskmnt/primary/CODEX/HTAN/20210921_HT293N1-S...
349,20210921_HT293N1-S1H3A1,2021-09-21_HT293N1_2,20210921,HT293N1,HT293N1-S1H3A1,NaN,/diskmnt/primary/CODEX/HTAN/20210921_HT293N1-S...


In [17]:
def get_geojson(bbox):
    bbox = re.split(r'[ ]?,[ ]?', bbox)
    bbox = [int(x) for x in bbox]
    r1, r2, c1, c2 = bbox
    d = {
        "type": "FeatureCollection",
        "features": [
            {
                "type": "Feature",
                "id": str(uuid.uuid4()),
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [
                        [
                            [c1, r1],
                            [c2, r2],
                            [c2, r1],
                            [c1, r2]
                        ]
                    ]
                }
            }
        ]
    }
    return d

In [18]:
fps = []
for i, row in df.iterrows():
    bbox = row['bbox']
    if not pd.isnull(bbox):
        fname = row['filepath']
        fname = re.sub(r'^(.*)\.(ome)?\.tiff?$', r'\1.geojson', fname)
        d = get_geojson(bbox)
        json.dump(d, open(fname, 'w'))
        fps.append(fname)
    else:
        fps.append('')
fps

['/diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1/HT913Z1-S2H1FP1_U2__20240812.geojson',
 '/diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1/HT913Z1-S2H1FP1_U12__20240812.geojson',
 '/diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1/HT913Z1-S2H1FP1_U22__20240812.geojson',
 '/diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1/HT913Z1-S2H1FP1_U41__20240812.geojson',
 '/diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_2/HT913Z1-S2H1FP1_U51__20240812.geojson',
 '/diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_2/HT913Z1-S2H1FP1_U52__20240812.geojson',
 '/diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_2/HT913Z1-S2H1FP1_U53__20240812.geojson',
 '/diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_2/HT913Z1-S2H1FP1_U54__20240812.geojson',
 '/diskmnt/primary/CODEX/HTAN/20240812_Prostate_s

In [16]:
for fp in fps:
    print(fp)

/diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1/HT913Z1-S2H1FP1_U2__20240812.geojson
/diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1/HT913Z1-S2H1FP1_U12__20240812.geojson
/diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1/HT913Z1-S2H1FP1_U22__20240812.geojson
/diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1/HT913Z1-S2H1FP1_U41__20240812.geojson
/diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_2/HT913Z1-S2H1FP1_U51__20240812.geojson
/diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_2/HT913Z1-S2H1FP1_U52__20240812.geojson
/diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_2/HT913Z1-S2H1FP1_U53__20240812.geojson
/diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_2/HT913Z1-S2H1FP1_U54__20240812.geojson
/diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_3/HT91